In [ ]:
#Results of YOLOv8 with unannotated lung images

In [2]:
%pip install ultralytics
# Importing necessary library for installing Ultralytics

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
from PIL import Image
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.1.35 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5519.7/8062.4 GB disk)


In [3]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.9 MB/s eta 0:00:0000:01


In [4]:
import wandb

# Replace '1c4e40a21a232ce25e3ddb9271911ef7bd2b70a8' with your actual W&B API key
wandb.login(key='1c4e40a21a232ce25e3ddb9271911ef7bd2b70a8')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

**YOLOv8 Labeling for Unannotated Lung Images
In this section, we provide a code snippet to generate YOLO labels for unannotated lung images using YOLOv8. The code takes images from a specified directory (image_directory) and creates YOLO label files in another directory (output_directory).

**Code Explanation:
*Import Libraries: We import the necessary os library for directory operations.

**Directories:

image_directory: Directory containing resized lung images.
output_directory: Directory where YOLO labels will be saved.
Directory Creation: We create the output directory if it doesn't exist using os.makedirs().

*Coordinates:

x_center, y_center, width, height: These are sample coordinates representing bounding box information. You can adjust these values based on your specific requirements or annotations.
File Iteration: We iterate through all files in the image_directory.

*Label Generation:

For each image file, we extract the class ID and create a YOLO label string.
The YOLO label format is class_id x_center y_center width height.
The label is saved in a .txt file with the same name as the image file in the output_directory.
Usage:
Replace image_directory with the path to your resized lung images.
Adjust the coordinates (x_center, y_center, width, height) as needed.
Run the code to generate YOLO labels for your unannotated lung images.
By using this code snippet, you can generate YOLO labels for unannotated lung images to prepare them for further YOLOv8 training or evaluation.


In [6]:
import os
image_directory = '/kaggle/working/resized_images'


# image_directory = '/kaggle/input/chest-xray-masks-and-labels/Lung Segmentation/CXR_png'

# Output directory for YOLO labels
output_directory = '/kaggle/working/yolo_labels'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Coordinates for YOLO labels
x_center, y_center, width, height = 0.50390625, 0.4828125, 0.90078125, 0.878125
# Iterate through all files in the directory
for filename in os.listdir(image_directory):
    # Check if the file is an image (you can add more conditions if needed)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Extract class ID from the filename
        class_id = int(filename.split('_')[-1].split('.')[0])

        # Define YOLO label text
        yolo_label = f"{class_id} {x_center} {y_center} {width} {height}"

        # Define the output label file path
        label_path = os.path.join(output_directory, filename.replace('.png', '.txt'))

        # Write YOLO label text to the file
        with open(label_path, 'w') as label_file:
            label_file.write(yolo_label)

print("YOLO labels created successfully for all images.")

YOLO labels created successfully for all images.


Detailed Explanation of the YOLOv8 Dataset Structure for Lung Images given on the read me part




In [18]:
import os
import random
import shutil

# Set the paths
input_directory = '/kaggle/working/resized_images'
output_directory = '/kaggle/working/yolo_labels'
train_directory = '/kaggle/working/train'
val_directory = '/kaggle/working/val'
test_directory = '/kaggle/working/test'

# Create output directories
os.makedirs(train_directory, exist_ok=True)
os.makedirs(val_directory, exist_ok=True)
os.makedirs(test_directory, exist_ok=True)

# Set the split percentages
train_percentage = 0.7
val_percentage = 0.15
test_percentage = 0.15

# Get the list of image files
image_files = [file for file in os.listdir(input_directory) if file.endswith('.png')]

# Shuffle the files for random splitting
random.shuffle(image_files)

# Calculate split indices
train_split = int(train_percentage * len(image_files))
val_split = int((train_percentage + val_percentage) * len(image_files))

# Copy files to respective directories
for i, file in enumerate(image_files):
    src_image = os.path.join(input_directory, file)
    src_label = os.path.join(output_directory, file.replace('.png', '.txt'))

    if i < train_split:
        dest_image = os.path.join(train_directory, file)
        dest_label = os.path.join(train_directory, file.replace('.png', '.txt'))
    elif i < val_split:
        dest_image = os.path.join(val_directory, file)
        dest_label = os.path.join(val_directory, file.replace('.png', '.txt'))
    else:
        dest_image = os.path.join(test_directory, file)
        dest_label = os.path.join(test_directory, file.replace('.png', '.txt'))

    shutil.copy(src_image, dest_image)
    shutil.copy(src_label, dest_label)

In [19]:
!cp /kaggle/input/lunglung5/lung2.yaml /kaggle/working


In [20]:
from ultralytics import YOLO
# 2. Choose our yaml file
yaml_filename = 'lung2.yaml' 

# 3. Create Yolo model
model = YOLO('yolov8n.yaml') # creates Yolo object from 'yolov8n.yaml' configuration file. 
model = YOLO('yolov8n.pt')   # Loads pretrained weights             
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# 4. Train the model
model.train(data='{}'.format(yaml_filename)) 

100%|██████████| 6.23M/6.23M [00:00<00:00, 162MB/s]


Transferred 355/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.1.35 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=lung2.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

100%|██████████| 755k/755k [00:00<00:00, 39.8MB/s]
2024-03-27 20:34:34,033	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-27 20:34:34,895	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-27 20:34:37.202122: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 20:34:37.202260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 20:34:37.330025: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS facto

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


...


100 epochs completed in 0.317 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  2.22it/s]


                   all        222        222      0.975      0.955      0.993      0.993
                healty        222        107          1      0.912      0.993      0.993
              unhealty        222        115       0.95      0.998      0.992      0.992
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train


lr/pg0,▃████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▃████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▅▁▆▇▇▆▇▆▇▇▇▇▇▇▇▇███▇████████████████████
metrics/mAP50-95(B),▅▁▅▆▇▆▇▆▇▇▇▇▇▆▇▇████████████████████████
metrics/precision(B),▁▁▄▅▆▃▇▅▄▅▄▃▆▄▇▄▇▇▇▆▇▇█▇▆▅▇▇▇█▇█▇▇██████
metrics/recall(B),▇▁▆▆▆▇▆▄▆▆▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████▇██▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x782125afc400>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [23]:

# Load a model
model = YOLO('yolov8n.pt')  # load an official model
model = YOLO('runs/detect/train/weights/best.pt')  # load a custom model

# Predict with the model
results = model('/kaggle/working/test/CHNCXR_0636_1.png')  # predict on an image


image 1/1 /kaggle/working/test/CHNCXR_0636_1.png: 640x640 1 unhealty, 6.8ms
Speed: 1.5ms preprocess, 6.8ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


In [25]:
# 5. Load the trained weights
model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

# 6. to run prediction on 1 image at a time
im = Image.open('/kaggle/working/test/CHNCXR_0636_1.png')
results = model.predict(source=im, save=True)


0: 640x640 1 unhealty, 9.4ms
Speed: 2.9ms preprocess, 9.4ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict
